In [1]:
# To test for PER, LOC and ORG Only

from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import os
import numpy as np

print(sklearn.__version__)

0.19.0


In [2]:
# Divide in train and test files [80:20] 

# Directory having content

doc_dir = 'Data/content'

train_file_list = []
test_file_list = []

for f in os.listdir(doc_dir):
    #Random Sampling
    if np.random.uniform(0,1)< 0.8:
        train_file_list.append(f)
    else:
        test_file_list.append(f)

print len(train_file_list)
print len(test_file_list)

1058
249


In [3]:
# Build Tagged dataset to be fed in nltk hmm tagger
# It is a list of tagged sentences. 
# training_data = [ [(word, tag), (word, tag).....]
# [(word, tag), (word, tag).....]
#]

# Directory having tags
tag_dir = 'Data/new_tags'

training_data = []

for f in train_file_list:
    training_sentences =[]
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    lines_in_word_file = []
    lines_in_tag_file = []
    with open(word_file_path, "rt") as word_file:
        for line in word_file:
            lines_in_word_file.append(line)
    with open(tag_file_path, "rt") as tag_file:
        for line in tag_file:
            lines_in_tag_file.append(line)
    if (len(lines_in_word_file) == len(lines_in_tag_file)) and len(lines_in_word_file) > 0:
        for i in xrange(len(lines_in_word_file)):
            word_in_file = lines_in_word_file[i].split()
            tag_in_file = lines_in_tag_file[i].split()
            pairs_in_line = []
            length = min(len(word_in_file), len(tag_in_file))
            #Create the word_tag pair
            for j in xrange(length):
                pairs_in_line.append((word_in_file[j], tag_in_file[j]));
            training_sentences.append(pairs_in_line)
    if len(training_sentences) > 0:
        training_data.extend(training_sentences)

#print training_data[0]

In [4]:
def word2features(sent, i):
    word = sent[i][0]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
    ]
    if i > 0:
        word1 = sent[i-1][0]
        tag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
           # '-1:tag1=' + tag1,
        ])
    else:
        features.append('BOS')
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
        ])
    else:
        features.append('EOS')
        
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2tags(sent):
    return [tag for word, tag in sent]

def sent2words(sent):
    return [word for word, tag in sent] 

In [5]:
print training_data[0]
print sent2features(training_data[0])[0]
print sent2tags(training_data[0])[0]
print sent2words(training_data[0])[0]

[('Five', 'O'), ('people', 'O'), ('were', 'O'), ('killed', 'O'), ('and', 'O'), ('over', 'O'), ('50', 'O'), ('injured', 'O'), ('in', 'O'), ('three', 'O'), ('blasts', 'O'), ('set', 'O'), ('off', 'O'), ('by', 'O'), ('insurgent', 'O'), ('outfit', 'O'), ('ULFA', 'ORG_Others'), ('here', 'O'), ('today', 'O'), ('hours', 'O'), ('before', 'O'), ('Union', 'O'), ('Home', 'O'), ('Minister', 'O'), ('P', 'PER_Others'), ('Chidambaram', 'PER_Others'), ("'s", 'O'), ('visit', 'O'), ('to', 'O'), ('review', 'O'), ('law', 'O'), ('and', 'O'), ('order', 'O'), ('situation', 'O'), ('in', 'O'), ('the', 'O'), ('state', 'O'), ('rocked', 'O'), ('by', 'O'), ('deadly', 'O'), ('blasts', 'O'), ('that', 'O'), ('left', 'O'), ('88', 'O'), ('dead', 'O'), ('two', 'O'), ('months', 'O'), ('ago', 'O'), ('.', 'O'), ('.', 'O'), ('.', 'O'), ('Three', 'O'), ('people', 'O'), ('were', 'O'), ('killed', 'O'), ('and', 'O'), ('35', 'O'), ('injured', 'O'), (',', 'O'), ('including', 'O'), ('four', 'O'), ('women', 'O'), (',', 'O'), ('when'

In [6]:
x_train = [sent2features(s) for s in training_data]
y_train = [sent2tags(s) for s in training_data]

In [7]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(x_train, y_train):
    trainer.append(xseq, yseq)

In [8]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [9]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [10]:
# Train and save the model
trainer.train('osint-bomb-blast.crfsuite.topcat')

In [11]:
tagger = pycrfsuite.Tagger()
tagger.open('osint-bomb-blast.crfsuite.topcat')

In [12]:
# Create test data
test_data = []
for f in test_file_list:
    test_sentences =[]
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    lines_in_word_file = []
    lines_in_tag_file = []
    with open(word_file_path, "rt") as word_file:
        for line in word_file:
            lines_in_word_file.append(line)
    with open(tag_file_path, "rt") as tag_file:
        for line in tag_file:
            lines_in_tag_file.append(line)
    if (len(lines_in_word_file) == len(lines_in_tag_file)) and len(lines_in_word_file) > 0:
        for i in xrange(len(lines_in_word_file)):
            word_in_file = lines_in_word_file[i].split()
            tag_in_file = lines_in_tag_file[i].split()
            pairs_in_line = []
            length = min(len(word_in_file), len(tag_in_file))
            #Create the word_tag pair
            for j in xrange(length):
                pairs_in_line.append((word_in_file[j], tag_in_file[j]));
            test_sentences.append(pairs_in_line)
    if len(test_sentences) > 0:
        test_data.extend(test_sentences)

print test_data[0]

[('The', 'O'), ('Maharashtra', 'ORG_Others'), ('Anti-Terrorism', 'ORG_Others'), ('Squad', 'ORG_Others'), ('has', 'O'), ('finally', 'O'), ('established', 'O'), ('the', 'O'), ('Sri', 'LOC_Others'), ('Lankan', 'LOC_Others'), ('connection', 'O'), ('in', 'O'), ('the', 'O'), ('Pune', 'LOC_Others'), ('German', 'LOC_Event'), ('Bakery', 'LOC_Event'), ('blast', 'O'), ('case', 'O'), ('.', 'O'), ('Mirza', 'PER_Accused'), ('Himayat', 'PER_Accused'), ('Inayat', 'PER_Accused'), ('Baig', 'PER_Accused'), (',', 'O'), ('the', 'O'), ('alleged', 'O'), ('mastermind', 'O'), (',', 'O'), ('underwent', 'O'), ('training', 'O'), ('to', 'O'), ('assemble', 'O'), ('explosive', 'O'), ('devices', 'O'), ('and', 'O'), ('cause', 'O'), ('their', 'O'), ('explosion', 'O'), ("'", 'O'), ('in', 'O'), ('Colombo', 'LOC_Others'), ('in', 'O'), ('March', 'O'), ('2008', 'O'), (',', 'O'), ('says', 'O'), ('the', 'O'), ('charge', 'O'), ('sheet', 'O'), ('filed', 'O'), ('by', 'O'), ('the', 'O'), ('ATS', 'O'), ('against', 'O'), ('Baig', '

In [13]:
example_sent = test_data[0]
print(' '.join(sent2words(example_sent)))

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2tags(example_sent)))

The Maharashtra Anti-Terrorism Squad has finally established the Sri Lankan connection in the Pune German Bakery blast case . Mirza Himayat Inayat Baig , the alleged mastermind , underwent training to assemble explosive devices and cause their explosion ' in Colombo in March 2008 , says the charge sheet filed by the ATS against Baig and six other accused . He was trained by two of the accused , Faiyaz Kagzi and Zabiuddin Ansari . The charge sheet says Kagzi and Ansari gave Baig money or funding the purchase of explosive devices and the travel of indoctrinated Muslim youth to Pakistan for training . After training in Pakistan , Baig returned to India and lived in the non-descript Udgir taluk in Maharashtra 's Latur district under concealed identity as Yusuf ' and Hasan ' . In the last week of January 2010 , on instructions from two of the accused Riyaz Bhatkal and Iqbal Bhatkal , the other two accused Yasin Bhatkal and Mohsin Chowdhary met Baig in Udgir to discuss , finalise and put int

In [14]:
def report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [15]:
x_test = [sent2features(s) for s in test_data]
y_test = [sent2tags(s) for s in test_data]

In [16]:
%%time
y_pred = [tagger.tag(xseq) for xseq in x_test]

CPU times: user 888 ms, sys: 4 ms, total: 892 ms
Wall time: 895 ms


In [17]:
print(report(y_test, y_pred))

             precision    recall  f1-score   support

    Accused       0.00      0.00      0.00        73
      Event       0.00      0.00      0.00        51
LOC_Accused       0.20      0.01      0.01       163
  LOC_Assoc       0.00      0.00      0.00        51
  LOC_Event       0.53      0.42      0.47      1151
 LOC_Others       0.58      0.54      0.56      1721
 LOC_Victim       0.00      0.00      0.00        20
   Location       0.33      0.02      0.04        95
ORG_Accused       0.77      0.76      0.76       572
  ORG_Assoc       0.00      0.00      0.00        10
 ORG_Others       0.65      0.56      0.60      2091
 ORG_Victim       0.00      0.00      0.00        65
PER_Accused       0.62      0.38      0.47       808
  PER_Assoc       0.00      0.00      0.00        64
 PER_Others       0.72      0.70      0.71      1769
 PER_Victim       0.38      0.24      0.30       306
     Victim       0.00      0.00      0.00         1

avg / total       0.60      0.51      0.55  

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
num_test_data = len(y_test)
print len(y_pred)
print num_test_data

249
249


In [19]:
# Test
actual_tag_count = dict()
matched_tag_count = dict()
matched_tag_count['PER'] = 0
actual_tag_count['PER'] = 0
matched_tag_count['ORG'] = 0
actual_tag_count['ORG'] = 0
matched_tag_count['LOC'] = 0
actual_tag_count['LOC'] = 0

for i in xrange(num_test_data):
    y_test_i = y_test[i]
    y_pred_i = y_pred[i]
    test_tag_length = len(y_test[i])
    predicted_tag_length = len(y_pred[i])
    for j in xrange(test_tag_length):
        if y_test_i[j][:3] in actual_tag_count.keys():
            actual_tag_count[y_test_i[j][:3]] = actual_tag_count[y_test_i[j][:3]] + 1
        
        if y_test_i[j][:3] == y_pred_i[j][:3]:
            if y_test_i[j][:3] in matched_tag_count.keys():
                matched_tag_count[y_test_i[j][:3]] = matched_tag_count[y_test_i[j][:3]] + 1

In [20]:
print('======= Accuracy Class Wise =====================\n')

print('Class    Matched Total %')
print('--------------------------')
print ('PER: '+str(matched_tag_count['PER'])+ ' ' + str(actual_tag_count['PER'])+ ' ' + str(matched_tag_count['PER']*100/actual_tag_count['PER'])+'%')
print ('ORG: '+str(matched_tag_count['ORG']) + ' '+ str(actual_tag_count['ORG'])+ ' '+str(matched_tag_count['ORG']*100/actual_tag_count['ORG'])+'%')
print ('LOC: '+str(matched_tag_count['LOC']) + ' '+ str(actual_tag_count['LOC'])+ ' '+str(matched_tag_count['LOC']*100/actual_tag_count['LOC'])+'%')

======= Accuracy Class Wise =====================

Class    Matched Total %
--------------------------
PER: 1766 2579 68%
ORG: 1629 2663 61%
LOC: 2094 3265 64%
